<a href="https://colab.research.google.com/github/tiarayosianti/GWR-Time/blob/main/GWR_Time_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import modules

Import the necessary modules

In [ ]:
!pip install mgwr
import numpy as np
import pandas as pd
from math import sqrt
from tqdm import tqdm
from pickle import TRUE
from mgwr.gwr import GWR
from mgwr.sel_bw import Sel_BW
import statsmodels.formula.api as smf
from statsmodels.compat import lzip
import statsmodels.stats.api as sms
from scipy.stats import norm, kstest
from sklearn.preprocessing import StandardScaler
from mgwr.diagnostics import get_CV, get_AIC
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.dates as dates

#Function

In [ ]:
# plot seasonal variabel respon
def seasonal_plot(city):
  fig, ax = plt.subplots(figsize=(10, 3))
  fig.autofmt_xdate()
  xfmt = dates.DateFormatter('%d-%m-%y')
  ax.xaxis.set_major_formatter(xfmt)
  df_city = data.loc[data['kota'] == city]
  df_city = df_city.set_index('Tanggal')
  df_city.index = pd.to_datetime(df_city.index)
  df_city = df_city.asfreq('D')
  sd_city = seasonal_decompose(df_city.PM10)
  ax.plot_date(df_city.index.to_pydatetime(), sd_city.seasonal, '-')
  start_date = mdates.datestr2num('2022-12-27')
  end_date = mdates.datestr2num('2024-01-02')
  ax.set_xlim(start_date, end_date)
  ax.xaxis.set_minor_locator(dates.WeekdayLocator(byweekday=(1), interval=1))
  ax.xaxis.set_minor_formatter(dates.DateFormatter('%d-%m-%y'))
  plt.setp(ax.xaxis.get_minorticklabels(), rotation=90, ha="right", fontsize=7)
  ax.xaxis.grid(True, which="minor")
  ax.yaxis.grid()
  plt.setp(ax.get_xticklabels(), visible=False)
  plt.yticks(fontsize=7)
  plt.title('Seasonal Component of PM$_{{10}}$ in {} City'.format(city))
  plt.xlabel('Date')
  plt.ylabel('Seasonality')
  plt.tight_layout()
  plt.show()

# Time related feature engineering with sine/cosine transformation
from sklearn.preprocessing import FunctionTransformer
def sin_transformer(period):
    return FunctionTransformer(lambda x: np.sin(x / period * 2 * np.pi))
def cos_transformer(period):
    return FunctionTransformer(lambda x: np.cos(x / period * 2 * np.pi))

# Split the dataset for train, validation, and test data
def split_data(city, dataset, train_frac, val_frac, test_frac, N_samples):
    location_current = dataset.loc[(dataset['kota'] == city)]
    location_current = location_current.reset_index()
    train, val, test = location_current.loc[:int(train_frac * N_samples-1), :], location_current.loc[int(train_frac * N_samples):int((val_frac + train_frac) * N_samples-1), :], location_current.loc[int((val_frac + train_frac) * N_samples):, :]
    return train, val, test

# Breusch-Pagan Test
def BP_test(df_train):
  fit = smf.ols('PM10 ~ Tavg + RH_avg + ff_avg + RR + week_sin + week_cos' , data = df_train).fit()
  # Conduct the Breusch-Pagan test
  labels = ['LM Statistic', 'LM-Test p-value', 'F-Statistic', 'F-Test p-value'] # LM: Lagrange multiplier
  # Get the test result
  test_result = sms.het_breuschpagan(fit.resid, fit.model.exog)
  if test_result[1]<0.05: result_hipotesis = 'Result: heteroscedasticity exists'
  return result_hipotesis, lzip(labels, test_result)

# Calibrate GWR model
def calibrate_gwr(coor, df_y, df_x, kernel, fixed, bw_min=None, bw_max=None):
  # Calibrate GWR model
  if fixed == False:
    # in adaptive bandwidth we are using min bandwidth and max bandwidth to prevent error
    gwr_selector = Sel_BW(coor, df_y, df_x, kernel=kernel, fixed=fixed, spherical=True)
    gwr_bw = gwr_selector.search(criterion="CV", bw_min=bw_min, bw_max=bw_max)
    gwr_model = GWR(coords=coor, y=df_y, X=df_x, bw=gwr_bw, kernel=kernel, fixed=fixed, spherical=True)
    gwr_results = gwr_model.fit()
  else:
    gwr_selector = Sel_BW(coor, df_y, df_x, kernel=kernel, fixed=fixed, spherical=True)
    gwr_bw = gwr_selector.search(criterion="CV")
    gwr_model = GWR(coords=coor, y=df_y, X=df_x, bw=gwr_bw, kernel=kernel, fixed=fixed, spherical=True)
    gwr_results = gwr_model.fit()
  # Result
  AIC = get_AIC(gwr_results)
  CV = get_CV(gwr_results)
  R2 = gwr_results.R2
  # p_vals_betas = gwr_results.spatial_variability(gwr_selector, 10)
  # Kernel
  kernel_used = None
  if fixed==True: kernel_used='Fixed '+kernel
  else: kernel_used='Adaptive '+kernel
  print('Kernel    : ', kernel_used,
        '\nBandwidth : ', gwr_bw,
        '\nCV        : ', np.around(CV,30),
        '\nAIC       : ', np.around(AIC,30),
        '\nR2        : ', np.around(R2,30))
  return gwr_model, gwr_results

# Residual Normality Test
def norm_kstest(resid):
  loc, scale = norm.fit(resid.resid_response)
  n = norm(loc=loc, scale=scale)
  return kstest(resid.resid_response, n.cdf)

# GWR Parameter
def params_b(gwr_result):
  params_b = pd.DataFrame(gwr_result.params, columns=['b0','b1','b2','b3','b4','b5','b6'])
  params_b.drop_duplicates(inplace=True)
  params_b.reset_index(drop=True, inplace=True)
  params_b.index = ['Jakarta Pusat', 'Jakarta Utara', 'Jakarta Selatan', 'Jakarta Timur', 'Jakarta Barat']
  return params_b

# t Statistics
def t_hitung(gwr_result):
  t_values = pd.DataFrame(gwr_result.tvalues, columns=['b0','b1','b2','b3','b4','b5','b6'])
  t_values.drop_duplicates(inplace=True)
  t_values.reset_index(drop=True, inplace=True)
  t_values.index = ['Jakarta Pusat', 'Jakarta Utara', 'Jakarta Selatan', 'Jakarta Timur', 'Jakarta Barat']
  return t_values

# Filtered t Statistics
def t_filtered(gwr_result):
  f_t_values = pd.DataFrame(gwr_result.filter_tvals(), columns=['b0','b1','b2','b3','b4','b5','b6'])
  f_t_values.drop_duplicates(inplace=True)
  f_t_values.reset_index(drop=True, inplace=True)
  f_t_values.index = ['Jakarta Pusat', 'Jakarta Utara', 'Jakarta Selatan', 'Jakarta Timur', 'Jakarta Barat']
  return f_t_values

# Local Multicolinearity
def local_coll(gwr_result):
  LCC, VIF, CN, VDP = gwr_result.local_collinearity()
  vif_bs = pd.DataFrame(VIF, columns=['X1','X2','X3','X4','X5','X6'])
  vif_bs.drop_duplicates(inplace=True)
  vif_bs.reset_index(drop=True, inplace=True)
  vif_bs.index = ['Jakarta Pusat', 'Jakarta Utara', 'Jakarta Selatan', 'Jakarta Timur', 'Jakarta Barat']
  return vif_bs

# Evaluation metrics
def evaluation_metrics(y_actual, y_predict):
  em_summary = pd.DataFrame(columns = ['MAPE', 'MAE', 'RMSE'])
  new_row = {
            'MAPE': mean_absolute_percentage_error(y_actual, y_predict)*100,
            'MAE': mean_absolute_error(y_actual, y_predict),
            'RMSE': sqrt(mean_squared_error(y_actual, y_predict))
          }
  em_summary.loc[0] = new_row
  return em_summary

#Preprocess data

##Import Data

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/new_df_clean_pm10_meteorologi.csv')
data

Here, the data has several columns such as Tanggal (date),	Tavg (temperature),	RH_avg (humidity),	ff_avg (wind speed),	RR (rainfall),	kota,	Latitude,	Longitude, and	PM10.

In [ ]:
data.info()

In [ ]:
data.kota.unique()

There are 5 cities data used, include: Jakarta Pusat, Jakarta Utara, Jakarta Selatan, Jakarta Timur, and Jakarta Barat.

##Scaling

In [ ]:
# see the mean and standar deviation before scaling
data_x = data[['Tavg',	'RH_avg',	'ff_avg',	'RR']]
print(data_x.mean(axis=0))
print(data_x.std(axis=0))

data_y_pm10 = data[['PM10']]
print(data_y_pm10.mean(axis=0))
print(data_y_pm10.std(axis=0))

In [ ]:
# Scaling data
scaler_x = StandardScaler()
scaler_y = StandardScaler()

data_x_scaled_weather = scaler_x.fit_transform(data_x)
print("X scaled shape :", data_x_scaled_weather.shape)
print("X variables mean :", data_x_scaled_weather.mean(axis=0))
print("X variables std :", data_x_scaled_weather.std(axis=0))

data_y_scaled_pm10 = scaler_y.fit_transform(data_y_pm10)
print("Y scaled shape :", data_y_scaled_pm10.shape)
print("Y variable mean :", data_y_scaled_pm10.mean(axis=0))
print("Y variable std :", data_y_scaled_pm10.std(axis=0))

data_x_scaled = pd.DataFrame(data_x_scaled_weather, columns = ['Tavg',	'RH_avg',	'ff_avg',	'RR'])
data_y_scaled = pd.DataFrame(data_y_scaled_pm10, columns = ['PM10'])

data_scaled = data[['Tanggal','kota', 'Latitude', 'Longitude']]
data_scaled = pd.concat([data_scaled, data_y_scaled, data_x_scaled], axis=1)
print("Data scaled shape :", data_scaled.shape)
data_scaled.head()

In [ ]:
data_scaled.info()

## Time related feature engineering

In [ ]:
cities = data.kota.unique()
for city in cities:
  seasonal_plot(city)

In [ ]:
# making weekly variables
weeks = pd.DatetimeIndex(data_scaled['Tanggal']).isocalendar().week.astype(np.int64)
week = pd.DataFrame(weeks.values,columns=["week"])
week['week_sin'] = sin_transformer(52).fit_transform(week)["week"]
week['week_cos'] = cos_transformer(52).fit_transform(week)["week"]
week.head()

In [ ]:
week.week.unique()

In [ ]:
data_ready = data_scaled[['Tanggal','kota', 'Latitude', 'Longitude', 'PM10', 'Tavg',	'RH_avg',	'ff_avg',	'RR']]
data_ready = pd.concat([data_ready, week], axis=1)
data_ready.drop(['week'], axis=1, inplace=True)
data_ready.head()

In [ ]:
data_ready.info()

##Split the dataset

In [ ]:
# Split the dataset
N_samples = 365
train_frac = 0.6
val_frac = 0.2
test_frac = 0.2

train_size = int(N_samples * train_frac)
val_size = int(N_samples * val_frac)
test_size = N_samples - train_size - val_size
print("Data train rows      :", train_size)
print("Data validation rows :", val_size)
print("Data test rows       :", test_size)

df_train = pd.DataFrame()
df_val = pd.DataFrame()
df_test = pd.DataFrame()
cities = data_scaled['kota'].unique()
print("\nCities :", cities,"\n")

for city in tqdm(cities):
    d_train, d_val, d_test = split_data(city, data_ready, train_frac, val_frac, test_frac, N_samples)
    df_train = pd.concat([df_train,d_train])
    df_val = pd.concat([df_val,d_val])
    df_test = pd.concat([df_test,d_test])

print("\n \nTrain shape :", df_train.shape)
print("Val shape :", df_val.shape)
print("Test shape :", df_test.shape)

In [ ]:
df_train.columns

In [ ]:
# reshape the data
train_X = np.array(df_train[['Tavg',	'RH_avg',	'ff_avg',	'RR', 'week_sin', 'week_cos']]).reshape(-1, 6)
val_X = np.array(df_val[['Tavg',	'RH_avg',	'ff_avg',	'RR', 'week_sin', 'week_cos'   ]]).reshape(-1, 6)
test_X = np.array(df_test[['Tavg',	'RH_avg',	'ff_avg',	'RR', 'week_sin', 'week_cos' ]]).reshape(-1, 6)

train_y = np.array(df_train['PM10']).reshape(-1, 1)
val_y = np.array(df_val['PM10']).reshape(-1, 1)
test_y = np.array(df_test['PM10']).reshape(-1, 1)

u_train = df_train['Longitude']
v_train = df_train['Latitude']
coords_train = list(zip(u_train,v_train))

u_val = df_val['Longitude']
v_val = df_val['Latitude']
coords_val = np.array(list(zip(u_val,v_val)))

u_test = df_test['Longitude']
v_test = df_test['Latitude']
coords_test = np.array(list(zip(u_test,v_test)))

print("Train shape :", train_X.shape, train_y.shape, len(coords_train))
print("Val shape :", val_X.shape, val_y.shape, len(coords_val))
print("Test shape :", test_X.shape, test_y.shape, len(coords_test))

In [ ]:
df_train

In [ ]:
df_val

In [ ]:
df_test

##Breuch Pagan test

In [ ]:
BP_test(df_train)

#Model GWR

##Adaptive bisquare

In [ ]:
model_pm10_ab, result_pm10_ab = calibrate_gwr(coor = coords_train,
                            df_y = train_y,
                            df_x = train_X,
                            kernel = 'bisquare',
                            fixed = False,
                            bw_min = 220,
                            bw_max = 500
                            )

##Fixed bisquare

In [ ]:
model_pm10_fb,result_pm10_fb = calibrate_gwr(coor = coords_train,
                            df_y = train_y,
                            df_x = train_X,
                            kernel = 'bisquare',
                            fixed = True
                            )

##Adaptive gaussian

In [ ]:
model_pm10_gs,result_pm10_gs = calibrate_gwr(coor = coords_train,
                            df_y = train_y,
                            df_x = train_X,
                            kernel = 'gaussian',
                            fixed = False,
                            bw_min = 220,
                            bw_max = 500
                            )

##Fixed gaussian

In [ ]:
model_pm10_fgs,result_pm10_fgs = calibrate_gwr(coor = coords_train,
                            df_y = train_y,
                            df_x = train_X,
                            kernel = 'gaussian',
                            fixed = True
                            )

##Adaptive exponential

In [ ]:
model_pm10_ex,result_pm10_ex = calibrate_gwr(coor = coords_train,
                            df_y = train_y,
                            df_x = train_X,
                            kernel = 'exponential',
                            fixed = False,
                            bw_min = 220,
                            bw_max = 500
                            )

##Fixed exponential

In [ ]:
model_pm10_fex, result_pm10_fex = calibrate_gwr(coor = coords_train,
                                  df_y = train_y,
                                  df_x = train_X,
                                  kernel = 'exponential',
                                  fixed = True
                                  )

In [ ]:
norm_kstest(result_pm10_fex)

In [ ]:
result_pm10_fex.summary()

In [ ]:
# parameters
params_b_pm10_fex = params_b(result_pm10_fex)
params_b_pm10_fex

In [ ]:
# t Statistics
t_hitung(result_pm10_fex)

In [ ]:
# t filter
t_filtered(result_pm10_fex)

In [ ]:
# t table
result_pm10_fex.critical_tval()

In [ ]:
# local multicolinearity
local_coll(result_pm10_fex)

In [ ]:
# evaluation metrics data train
yhats_inv_pm10_train_fex = scaler_y.inverse_transform(result_pm10_fex.predy)
train_y_inv_pm10_fex = scaler_y.inverse_transform(train_y)
evaluation_metrics(train_y_inv_pm10_fex, yhats_inv_pm10_train_fex)

In [ ]:
scale_fex = result_pm10_fex.scale
residuals_fex = result_pm10_fex.resid_response

# predict with validation data
results_val_fex = model_pm10_fex.predict(coords_val, val_X, scale_fex, residuals_fex)
predicted_y_val_fex = results_val_fex.predictions

# evaluation metrics data validation
yhats_inv_pm10_val_fex = scaler_y.inverse_transform(predicted_y_val_fex)
val_y_inv_pm10_fex = scaler_y.inverse_transform(val_y)
evaluation_metrics(val_y_inv_pm10_fex, yhats_inv_pm10_val_fex)

In [ ]:
# predict with test data
results_test_fex = model_pm10_fex.predict(coords_test, test_X, scale_fex, residuals_fex)
predicted_y_test_fex = results_test_fex.predictions

# evaluation metrics data test
yhats_inv_pm10_test_fex = scaler_y.inverse_transform(predicted_y_test_fex)
test_y_inv_pm10_fex = scaler_y.inverse_transform(test_y)
evaluation_metrics(test_y_inv_pm10_fex, yhats_inv_pm10_test_fex)

#End of notebook